In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

import scipy
from sklearn.grid_search import RandomizedSearchCV

/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2

# Read Dataset

In [3]:
f = open('datasets/conll2003/train.txt', 'r')
lines = f.readlines()
f.close()

del lines[0]
del lines[0]

dataset = []
sentence = []
for line in lines:
    splitter = line.strip().split(' ')
    if splitter[0] == '':
        continue
    elif (splitter[0] == '-DOCSTART-'):
        dataset.append(sentence)
        sentence = []
    else:
        token = splitter[0]
        tag = splitter[3]
        sentence.append((token, tag))

In [4]:
def convert_conlltxt2dataset(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    
    del lines[0]
    del lines[0]
    
    dataset = []
    sentence = []
    for line in lines:
        splitter = line.strip().split(' ')
        if splitter[0] == '':
            continue
        elif (splitter[0] == '-DOCSTART-'):
            dataset.append(sentence)
            sentence = []
        else:
            token = splitter[0]
            tag = splitter[3]
            sentence.append((token, tag))
    return dataset

In [5]:
validation_dataset = convert_conlltxt2dataset('datasets/conll2003/valid.txt')
test_dataset = convert_conlltxt2dataset('datasets/conll2003/test.txt')

# Add Postag to Dataset

## Example

In [6]:
postagger = StanfordPOSTaggerWrapper()
postag = postagger.tag('+44 171')
postag

[('+44', 'CD'), ('171', 'CD')]

In [7]:
def add_postag2dataset(dataset):
    postagger = StanfordPOSTaggerWrapper()
    dataset_with_postag = []
    for sent in dataset:
        postagged_sent = []
        for index, (token, tag) in enumerate(sent):
            postagged_token = postagger.tag(token)
            postagged_sent.append((token, postagged_token[0][1], tag))
        dataset_with_postag.append(postagged_sent)
        
    return dataset_with_postag

postagged_validation_dataset = add_postag2dataset(validation_dataset)
postagged_test_dataset = add_postag2dataset(test_dataset)

# Delete Unused Dataset
del validation_dataset
del test_dataset

# Extract Feature

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    # Ortographic Feature, Word, POSTag & N-Gram
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isupper()': word.isupper(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }
    
    # Position
    features.update({
        'pos_front': i,
        'pos_end': len(sent) - i
    })
    
    # Bag Of Words
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2postag(sent):
    return [postag for token, postag, label in sent]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# Feature Extraction

In [9]:
X_val = [sent2features(sent) for sent in postagged_validation_dataset]
y_val = [sent2labels(sent) for sent in postagged_validation_dataset]

X_test = [sent2features(sent) for sent in postagged_test_dataset]
y_test = [sent2labels(sent) for sent in postagged_test_dataset]

del postagged_validation_dataset
del postagged_test_dataset

# Load Word Embedding and Add to The Feature

In [10]:
def load_word_embedding_model(filename):
    f = open(filename, 'r', encoding='utf-8')
    lines = f.readlines()
    del lines[0]

    embedding_dict = {}
    counter = 0
    for line in lines:
        line = line.strip()
        arr = line.split(' ')
        word = arr[0].lower()
        vectors = [float(x) for x in arr[1:]]
        embedding_dict[word] = vectors
    return embedding_dict

def add_word_embedding2feature(embedding_dict, feature):
    VECTOR_SPACE_SIZE = 50
    for i, sentence in enumerate(feature):
        for j, token in enumerate(sentence):
            word = token['word.lower()']
            vector = []
            if word in embedding_dict:
                vector = embedding_dict[word]
            else:
                vector = [0 for i in range(VECTOR_SPACE_SIZE)]
            for k in range(len(vector)):
                wk = 'w{}'.format(k+1)
                feature[i][j][wk] = vector[k]
    return feature

from copy import deepcopy

In [11]:
cbow_embedding_dict = load_word_embedding_model('models/word_embedding/cbow_model.txt')
X_cbow = deepcopy(X_test)
X_cbow = add_word_embedding2feature(cbow_embedding_dict, X_cbow)

X_val_cbow = deepcopy(X_val)
X_val_cbow = add_word_embedding2feature(cbow_embedding_dict, X_val_cbow)

del cbow_embedding_dict

In [12]:
sg_embedding_dict = load_word_embedding_model('models/word_embedding/sg_model.txt')
X_sg = deepcopy(X_test)
X_sg = add_word_embedding2feature(sg_embedding_dict, X_sg)

X_val_sg = deepcopy(X_val)
X_val_sg = add_word_embedding2feature(sg_embedding_dict, X_val_sg)

del sg_embedding_dict

In [13]:
fasttext_embedding_dict = load_word_embedding_model('models/word_embedding/fasttext_model.txt')
X_fasttext = deepcopy(X_test)
X_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_fasttext)

X_val_fasttext = deepcopy(X_val)
X_val_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_val_fasttext)

del fasttext_embedding_dict

In [14]:
del X_test

In [15]:
X_cbow[0][0]

{'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 1.576941,
 'w2': -2.1714623,
 'w3': -1.0200785,
 'w4': -1.5237675,
 'w5': 1.7636915,
 'w6': 3.1906004,
 'w7': -2.6801944,
 'w8': -0.03286817,
 'w9': 2.877698,
 'w10': -2.67781,
 'w11': -2.0102458,
 'w12': 0.09859429,
 'w13': 0.6865189,
 'w14': -0.08337563,
 'w15': -3.2216864,
 'w16': 0.55069566,
 'w17': -3.327653,
 'w18': -2.3479667,
 'w19': -1.281957,
 'w20': 0.5317431,
 'w21': -1.3012091,
 'w22': 0.8570002,
 'w23': 0.1130651,
 'w24': -2.905357,
 'w25': 0.43759724,
 'w26': -0.42849925,
 'w27': -1.0062445,
 'w28': 0.05757309,
 'w29': -0.93642056,
 'w30': -0.35310736,
 'w31': 0.38900128,
 'w3

In [16]:
X_sg[0][0]

{'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 0.28394398,
 'w2': -0.28532466,
 'w3': -1.124571,
 'w4': -0.4304585,
 'w5': 0.004001268,
 'w6': 0.93360204,
 'w7': -0.20814724,
 'w8': 0.07862179,
 'w9': -0.40074977,
 'w10': -1.340784,
 'w11': 0.057163946,
 'w12': -0.020577392,
 'w13': 0.47397217,
 'w14': 0.5548938,
 'w15': -0.6635612,
 'w16': 0.06970437,
 'w17': -0.064371176,
 'w18': 0.67999274,
 'w19': -0.8801622,
 'w20': -0.50367016,
 'w21': -0.12410352,
 'w22': -0.41909856,
 'w23': -1.0188217,
 'w24': -0.41626242,
 'w25': 0.7018295,
 'w26': 0.07536637,
 'w27': -1.5404706,
 'w28': -0.26609275,
 'w29': -0.597601,
 'w30': 0.3595494,
 'w31'

In [17]:
X_fasttext[0][0]

{'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 0.56632817,
 'w2': -0.010743908,
 'w3': 0.28308353,
 'w4': 0.20814908,
 'w5': 0.13549374,
 'w6': -0.5231039,
 'w7': -0.12281269,
 'w8': -1.0767348,
 'w9': 0.42689112,
 'w10': -0.038685467,
 'w11': 0.0714525,
 'w12': 0.34661636,
 'w13': 0.070119075,
 'w14': 0.032985326,
 'w15': 0.24295594,
 'w16': 0.013151563,
 'w17': -0.6915679,
 'w18': -0.13650842,
 'w19': -0.3612757,
 'w20': 0.05955878,
 'w21': -0.686944,
 'w22': -0.10490251,
 'w23': -0.29388943,
 'w24': -0.048325066,
 'w25': -0.09750202,
 'w26': 0.010376323,
 'w27': -0.20131382,
 'w28': -1.0787536,
 'w29': 0.1044753,
 'w30': 0.1301,
 'w31

# Load Model

In [18]:
import pickle

In [19]:
filename_cbow = 'temp_crf_cbow.sav'
loaded_model_cbow = pickle.load(open(filename_cbow, 'rb'))

filename_sg = 'temp_crf_sg.sav'
loaded_model_sg = pickle.load(open(filename_sg, 'rb'))

filename_fasttext = 'temp_crf_fasttext.sav'
loaded_model_fasttext = pickle.load(open(filename_fasttext, 'rb'))

In [20]:
labels = list(loaded_model_cbow.classes_)
labels.remove('O')
labels

['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

In [21]:
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# Cross Val Score

In [22]:
x_val_score_cbow = cross_val_score(loaded_model_cbow, X_val_cbow, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_cbow)
print(x_val_score_cbow.mean())

[0.8262092  0.69083431 0.82868015 0.81987289 0.69118589]
0.7713564867348932


In [23]:
x_val_score_sg = cross_val_score(loaded_model_sg, X_val_sg, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_sg)
print(x_val_score_sg.mean())

[0.81696421 0.67704105 0.82677009 0.81192716 0.68955985]
0.7644524713996107


In [24]:
x_val_score_fasttext = cross_val_score(loaded_model_fasttext, X_val_fasttext, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_fasttext)
print(x_val_score_fasttext.mean())

[0.81848733 0.68621914 0.83599056 0.82169773 0.6925257 ]
0.7709840935136203


# CBOW

In [25]:
y_pred_cbow = loaded_model_cbow.predict(X_cbow)
metrics.flat_f1_score(y_test, y_pred_cbow,
                      average='weighted', labels=labels)

0.8186905218167554

# SG

In [26]:
y_pred_sg = loaded_model_sg.predict(X_sg)
metrics.flat_f1_score(y_test, y_pred_sg,
                      average='weighted', labels=labels)

0.8079198001068413

# FastText

In [27]:
y_pred_fasttext = loaded_model_fasttext.predict(X_fasttext)
metrics.flat_f1_score(y_test, y_pred_fasttext,
                      average='weighted', labels=labels)

0.8147009093053825